# Recurrent Neural Network Language Model

In this part, you'll implement a recurrent neural network language model (RNNLM) in TensorFlow. This class of models represents the cutting edge in language modeling, and your model will include many of the same features as the state-of-the-art.

As a reference, you may want to review the following:

- [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) (Andrej Karpathy, 2015)
- [Understanding LSTM Networks](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) (Chris Olah, 2015)

The specific model we'll build is based on the following papers. You should skim these (particularly the first one), but you don't need to read them in detail:

- [Recurrent neural network based language model](http://www.fit.vutbr.cz/research/groups/speech/publi/2010/mikolov_interspeech2010_IS100722.pdf) (Mikolov, et al. 2010)
- [Exploring the Limits of Language Modeling](http://arxiv.org/pdf/1602.02410.pdf) (Jozefowicz, et al. 2016)

We'll build our model entirely in TensorFlow, so you may want to review the [TensorFlow section of assignment 1](../a1/tensorflow/tensorflow.ipynb). We'll build our model in a similar style to the [NPLM demo](../../materials/nplm/nplm.ipynb). (_Unlike in A2, we won't use the Estimator API for this assignment._)

Finally, you'll want to frequently consult the [TensorFlow API reference](https://www.tensorflow.org/versions/master/api_docs/python/), and pay special attention to the types, dimensions and order of arguments for each function.  As we suggested you do in Assignment 1, you'll want to **draw the shape of any matrices you work with on a scrap paper** or you may have trouble keeping track of your forward path!

In [1]:
import tensorflow as tf
assert(tf.__version__.startswith("1."))

# Assignment Overview

Since the model for this assignment is a bit more complex, we've split it out into a writeup (this page), and a separate working notebook [rnnlm-workbook.ipynb](rnnlm-workbook.ipynb). You'll also be writing code in [rnnlm.py](rnnlm.py).

There are five parts, with most of the work in (a) and (b):

- **(a)** RNNLM Inputs and Parameters
- **(b)** Implementing the RNNLM
- **(c)** Training your RNNLM
- **(d)** Sampling Sentences
- **(e)** Linguistic Properties

**DO NOT EDIT THIS NOTEBOOK** - we will only grade [rnnlm-workbook.ipynb](rnnlm-workbook.ipynb) and [rnnlm.py](rnnlm.py).

# RNNLM Model Structure

![RNNLM](images/rnnlm_layers.png)

Here's the basic spec for our model. We'll use the following notation:

- $w^{(i)}$ for the $i^{th}$ word of the sequence (as an integer index)
- $x^{(i)}$ for the vector representation of $w^{(i)}$
- $h^{(i)}$ for the $i^{th}$ hidden state, with indices as in Section 5.8 of the async
- $o^{(i)}$ for the $i^{th}$ output state, which may or may not be the same as the hidden state
- $y^{(i)}$ for the $i^{th}$ target word, which for a language model is always equal to $w^{(i+1)}$

Let $ h^{(-1)} = h^{init} $ be an initial state. For an input sequence of $n$ words and $i = 0, ..., n-1$, we have:

- **Embedding layer:** $ x^{(i)} = W_{in}[w^{(i)}] $
- **Recurrent layer:** $ (o^{(i)}, h^{(i)}) = \text{CellFunc}(x^{(i)}, h^{(i-1)}) $
- **Output layer:** $\hat{P}(y^{(i)}) = \hat{P}(w^{(i+1)}) = \text{softmax}(o^{(i)}W_{out} + b_{out}) $
 
$\text{CellFunc}$ can be an arbitrary function representing our recurrent cell - it can be a simple RNN cell, or something more complicated like an LSTM, or even a stacked multi-layer cell. *Note that the cell has its own internal, trainable parameters.*

It may be convenient to deal with the logits of the output layer, which are the un-normalized inputs to the softmax:

$$ \text{logits}^{(i)} = o^{(i)}W_{out} + b_{out} $$

We'll use these as shorthand for important dimensions:
- `V` : vocabulary size
- `H` : hidden state size = embedding size = per-cell output size

## Multi-Layer Cells

One popular technique for improving the performance of RNNs is to stack multiple layers. Conceptually, this is similar to an ordinary multi-layer network, such as those you implemented on Assignment 1.

![RNNLM - multicell](images/rnnlm_multicell.png)

**Recurent layer 1** will take embeddings $ x^{(i)} $ as inputs and produce outputs $o^{(i)}_0$. We can feed these in to **Recurrent layer 2**, and get another set of outputs $o^{(i)}_1$, and so on. Note that because the input dimension of an RNN cell is typically the same as the output, all of these layers will have the same shape.

In TensorFlow, multi-layer RNNs are usually implemented using `tf.nn.rnn_cell.MultiRNNCell`. Instead of building the whole model layer-by-layer, `MultiRNNCell` returns a vertically-stacked cell, as shown by the dashed green lines above.

## Batching and Truncated Backpropagation Through Time (BPTT)

Batching for an RNN works the same as for any neural network: we'll run several copies of the RNN simultaneously, each with their own hidden state and outputs. Most TensorFlow functions are batch-aware, and expect `batch_size` as the first dimension.

With RNNs, however, we also need to consider the sequence length. In theory, we model our RNN as operating on sequences of arbitary length, but in practice it's much more efficient to work with batches where all the sequences have the same (maximum) length. TensorFlow calls this dimension `max_time`.  _Note: since LSTMs model sequences, a lot of the nomenclature around them mentions "time".  Whenever you see a reference to "time" in documentation, just read it as "word sequence position(s)"._

Put together, it looks like this, where our inputs $w$ and targets $y$ will both be 2D arrays of shape `[batch_size, max_time]`.

![RNNLM - batching](images/rnnlm_batching.png)

Note that along the batch dimension, sequences are independent. Along the time dimension, the output of one timestep is fed into the next. 

In the common case of processing sequences longer than `max_time`, we can chop the input up into smaller chunks, and carry the final hidden state from one batch as the input to the next. For example, given the input `[a b c d e f g h]` and `max_time = 4`, we would run twice:
```
h_init    -> RNN on [a b c d] -> h_final_0
h_final_0 -> RNN on [e f g h] -> h_final_1
```
We can also do this with batches, taking care to construct our batches in such a way that each batch lines up with it's predecessor. For example, with inputs `[a b c d e f g h]` and `[s t u v w x y z]`, we would do:
```
h_init    -> RNN on [a b c d] -> h_final_0
                    [s t u v]

h_final_0 -> RNN on [e f g h] -> h_final_1
                    [w x y z]
```
where our hidden states `h_init`, etc. have shape `[batch_size, state_size]`. (*Note that `state_size = H` for a simple RNN, but is larger for LSTMs or stacked cells.*)

Training in this setting is known as *truncated backpropagation through time*, or truncated BPTT. We can backpropagate errors within a batch for up to `max_time` timesteps, but not any further past the batch boundary. In practice with `max_time` greater than 20 or so, this doesn't significantly hurt the performance of our language model.

## (a) RNNLM Inputs and Parameters

**(written - no code)** Write your answers in the working notebook, [rnnlm-workbook.ipynb](rnnlm-workbook.ipynb). Please give brief answers - you shouldn't need more than a sentence or two for each part.

You should use big-O notation when appropriate (i.e. computing $\exp(\mathbf{v})$ for a vector $\mathbf{v}$ of length $n$ is $O(n)$ operations).  Assume for problems a(1-5) that:   
> Cell is one layer,  
> the embedding feature length and hidden-layer feature lengths are both H, and   
> ignore for the moment batch and max_time dimensions.  

1. Let $\text{CellFunc}$ be a simple RNN (__not an LSTM!__) cell (see async Section 5.8). Write the cell equation in terms of nonlinearities and matrix multiplication. How many parameters (matrix or vector elements) are there for this cell, in terms of `V` and `H`?
<p>
2. How many parameters are in the embedding layer? In the output layer? (By parameters, we mean total number of matrix elements across all train-able tensors. A $m \times n$ matrix has $mn$ elements.)
<p>
3. How many calculations (floating point operations) are required to compute $\hat{P}(w^{(i+1)})$ for a given *single* target word $w^{(i+1)}$, assuming $w^{(i)}$ given and $h^{(i-1)}$ already computed? How about for *all* target words?
<p>
4. How does your answer to 3. change if we approximate $\hat{P}(w^{(i+1)})$ with a sampled softmax with $k$ samples? How about if we use a hierarchical softmax? (*Recall that hierarchical softmax makes a series of left/right decisions using a binary classifier $P_s(\text{right}) = \sigma(u_s \cdot o^{(i)} + b_s)$ at each split $s$ in the tree.*)
<p>
5. If you have an LSTM with $H = 200$ and use sampled softmax with $k = 100$, what part of the network takes up the most computation time during training? (*Choose "embedding layer", "recurrent layer", or "output layer"*.)

Note: for $A \in \mathbb{R}^{m \times n}$ and $B \in \mathbb{R}^{n \times l}$, computing the matrix product $AB$ takes $O(mnl)$ time.

## (b) Implementing the RNNLM

In order to better manage the model parameters, we'll implement our RNNLM in the `RNNLM` class in `rnnlm.py`. We've given you a skeleton of starter code for this, but the bulk of the implementation is left to you.

Particularly, you'll need to implement three functions:
- `BuildCoreGraph()` : the main RNN itself
- `BuildTrainGraph()` : the training operations, including `train_loss_`, and `train_step_`
- `BuildSamplerGraph()` : operations to generate output samples (`pred_samples_`)

The starter code will define:

- `input_w_` : `[batch_size, max_time]` of `int32`. Input indices $w^{(i)}$ for each index $i$ in each sequence in the batch.
- `target_y_` : `[batch_size, max_time]` of `int32`. Target indices $y^{(i)}$ for each index $i$ in each sequence in the batch.

Your code will need to define the following variables and ops:
- `W_in_` : $ W_{in} $ for the embedding layer
- `cell_` : $ \text{CellFunc} $, which hides some internal variables
- `initial_h_` : $ h^{(-1)} = h^{init} $, as returned by `cell_zero_state(...)`.
- `final_h_` : $ h^{final} $, the output of the last cells in each batch.
- `W_out_` : $ W_{out} $ for the output layer
- `b_out_` : $ b_{out} $ for the output layer
- `loss_` : (scalar) *mean* cross-entropy loss across all indices in the entire batch
- `train_loss_` : (scalar) approximation of `loss_` using sampled softmax
- `train_step_` : training op (see Assignment 1)
- `pred_samples_` : `[batch_size, max_time, 1]` of `int32`, predicted samples for each batch element and each timestep.

Before you start, you should **write down the shapes of each tensor in terms of `V` and `H`**. Note that persistent variables (i.e. your model parameters) should **never** depend on `batch_size` or `max_time`.

**Hint:**  Many tensors will be three dimensional as follows:
![Three Dimensional Shape](common_shape.png)

Also note that everything should be a class variable, so `foo_` above would be referenced in the code as `self.foo_`.

See `rnnlm.py` for more documentation.

### Notes and Tips

You may find the following API functions useful:
- [tf.nn.rnn_cell](https://www.tensorflow.org/api_docs/python/tf/nn/rnn_cell) (particularly `cell.zero_state`)
- [tf.nn.dynamic_rnn](https://www.tensorflow.org/versions/master/api_docs/python/tf/nn/dynamic_rnn)
- [tf.nn.sparse_softmax_cross_entropy_with_logits](https://www.tensorflow.org/versions/master/api_docs/python/tf/nn/sparse_softmax_cross_entropy_with_logits)
- [tf.nn.sampled_softmax_loss](https://www.tensorflow.org/versions/master/api_docs/python/tf/nn/sampled_softmax_loss)
- [tf.multinomial](https://www.tensorflow.org/versions/master/api_docs/python/tf/multinomial)
- [tf.nn.embedding_lookup](https://www.tensorflow.org/versions/master/api_docs/python/tf/nn/embedding_lookup)

Additionally, you can expect to make heavy use of [tf.shape](https://www.tensorflow.org/versions/master/api_docs/python/tf/shape) and [tf.reshape](https://www.tensorflow.org/versions/master/api_docs/python/tf/reshape). Note especially that you can use `[-1]` as a dimension in `tf.reshape` to automatically infer the size. For example:
```python
x = tf.zeros([5,10], dtype=tf.float32)
x.reshape([-1,])    # shape [50,]
x.reshape([1, -1])  # shape [1, 50]
```
Or in a more specific example:
```python
# reshape to 2D, flattening first dimensions
X_r = tf.reshape(X, [-1,d])
Y_r = tf.some_function_on_rows(X_r)  
# reshape back to 3D, possibly with different last dimension
Y = tf.reshape(Y_r, [batch_size, max_time, -1])
```

#### `BuildCoreGraph`
We've implemented `MakeFancyRNNCell` for you, as a wrapper to construct LSTM cells with (optional) dropout and multi-layer cells.

You should use **`tf.nn.dynamic_rnn`** to build your recurrent layer. It takes care of making the recurrent connections and ensuring that the computation is done in the right (temporal) order, and gives you a nice wrapper that can take inputs of shape `[batch_size, max_time, ...]`.

You'll need to provide initializations for your variables in the embedding layer and the output layer. You should initialize weight and embedding matricies to uniform [-1.0,1.0] noise, and initialize all bias terms to zeros. The `tf.nn.rnn_cell` functions will automatically handle initialization of the internal cell variables (i.e. the LSTM matricies).

**_Hint:_** Look at the [NPLM notebook](../../materials/nplm/nplm.ipynb) for how to construct an embedding layer.

**_Hint:_** Constructing a RNN in TensorFlow involves two steps:

- Create a "template" LSTM cell. The `MakeFancyRNNCell` function in the starter code does this for you.  (Just pass `self.dropout_keep_prob_`, which we define for you, as the second parameter.)  _Optional: See [the original paper](https://www.cs.toronto.edu/~hinton/absps/JMLRdropout.pdf) if you want to learn more about dropout._

- Repeat that cell the appropriate number of times (`tf.nn.dynamic_rnn` does this for you).

**_Hint:_** There are a number of types of RNN cell.  LSTM is but one of them. What they all have in common is that they generate:

1. An "output" state you can use for classification, etc. For a LSTM, this is just the "H"idden layer.
2. Some "hidden" state that must be forwarded to the next cell in the sequence.  For a LSTM, this is a an [`LSTMStateTuple`](https://www.tensorflow.org/versions/master/api_docs/python/tf/contrib/rnn/LSTMStateTuple) containing the cell state ("C") and the hidden state ("H"). (_Don't worry about this type yourself - the cell from `MakeFancyRNNCell` will handle the return types! Just be sure to feed it in in the correct place._)

`tf.nn.dynamic_rnn` returns:

- The output layer (1 above) from each cell through the sequence.
- The final hidden state (2 above, from the last cell in the sequence).

**_Hint:_** No need to do the actual softmax in the output layer, just compute the logits. The loss function will handle the softmax internally.


**_Hint:_** As with assignment 1, be careful with the order and type (logits vs. labels) of parameters to tf.nn.sparse_softmax_cross_entropy_with_logits.


#### `BuildTrainGraph`
You implemented the softmax loss in `BuildCoreGraph`, but to speed up training, you should implement an approximate loss function for `train_loss_`. Use a sampled softmax with $K = 200$. Be sure that you use this approximate loss in your training code (part (c)), or your model will be very slow!
      
**_Hint:_** use `print` and `my_tensor_.get_shape()` here to make sure you understand the shape of all your variables.  Depending on how you implement the rest of the assignment, it's quite likely that you'll want to use a `tf.transpose` or `tf.expand_dims` or `tf.reshape`.

**_Hint:_**: In particular, the sampled softmax loss function does not accept the three dimensional `batch x sequence x final_hidden_layer` shape, so you'll need to reshape both the tensor holding the output hidden layers and the labels so that you are making only one prediction per row (hint, you'll need `batch x sequence` rows).

#### Choice of Optimizers

For training steps, you can use any optimizer, but we recommend `tf.train.AdamOptimizer` with gradient clipping (`tf.clip_by_global_norm`).  Adam adjusts the learning rate on a per-variable basis, and also adds a "momentum" term that improves the speed of convergence. See [An overview of gradient descent optimization algorithms](http://ruder.io/optimizing-gradient-descent/) for more.

For training with AdamOptimizer, you want to use the `learning_rate = 0.01` as defined under "Training Parameters" (next to batch size, num epochs, etc.). If you use `learning_rate = 0.1` with Adam, the model will likely overfit or training may be unstable. (However, 0.1 works well with Adagrad and vanilla SGD.)

Note that in the test module `rnnlm_test.py`, the learning rate on line 106  has been set to `0.01` according to the default suggestion of `AdamOptimizer`, if you wish to experiment with Adagrad or Vanilla SGD, please update this yourself.

## (c) Training your RNNLM

Follow the instructions for this step in the working notebook, [rnnlm-workbook.ipynb](rnnlm-workbook.ipynb). You'll need to:

1. Finish the implementation of the `run_epoch` function.
2. Finish the training code and train your model.

You should be able to get a perplexity score below 200 with a few minutes of training (3-5 epochs).

## (d) Sampling Sentences

If you didn't already in **part (b)**, implement the `BuildSamplerGraph()` method in `rnnlm.py` See the function docstring for more information.

Follow the instructions in the working notebook, [rnnlm-workbook.ipynb](rnnlm-workbook.ipynb). You'll implement the `sample_step()` method, and run it to generate some sentences from your model.

## (e) Linguistic Properties

Follow the instructions in the working notebook, [rnnlm-workbook.ipynb](rnnlm-workbook.ipynb). You'll use the `score_seq()` function to compare your model's score on several test sentences.